In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Get strip array from txt file
strips = np.genfromtxt("SBND_CRTpositionsSiPM-nogaps.txt", delimiter=None)

In [2]:
# Debug by printing
# Column  0   is strip ID
# Columns 1-3 are [x, y, z] positions in building coordinates
print(strips)

[[9.50000000e+03 3.31000000e+02 1.04625000e+03 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.50100000e+03 3.31000000e+02 1.04625000e+03 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.50200000e+03 3.31000000e+02 1.04625000e+03 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [8.32900000e+03 8.60881213e+02 1.00000000e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.33000000e+03 8.60981213e+02 1.00000000e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.33100000e+03 8.66831226e+02 1.00000000e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [3]:
# Just learning how to use python
print(type(strips))
print(strips.shape)

<class 'numpy.ndarray'>
(768, 7)


In [4]:
# Remove columns 4 onwards, not needed
reduced_strips = strips[:,0:4]

In [5]:
# Check I did it right
print(reduced_strips.shape)

(768, 4)


In [6]:
# And again...
print(reduced_strips)

[[9.50000000e+03 3.31000000e+02 1.04625000e+03 4.05000000e+00]
 [9.50100000e+03 3.31000000e+02 1.04625000e+03 9.90000000e+00]
 [9.50200000e+03 3.31000000e+02 1.04625000e+03 1.00000000e+01]
 ...
 [8.32900000e+03 8.60881213e+02 1.00000000e+02 2.76000000e+02]
 [8.33000000e+03 8.60981213e+02 1.00000000e+02 2.76000000e+02]
 [8.33100000e+03 8.66831226e+02 1.00000000e+02 2.76000000e+02]]


In [7]:
# The conversion to our usual detector coordinates that I understand
# First swap y & z
def Swap(arr, start_index, last_index):
    arr[:, [start_index, last_index]] = arr[:, [last_index, start_index]]
    
Swap(reduced_strips, 2, 3)

# Invert x-axis
reduced_strips = reduced_strips * [1, -1, 1, 1]

# Transform origin position (and add 300 to mac5 to match those in Igor's slides)
reduced_strips = reduced_strips + [30000, 471, -395, -265]

In [8]:
# Check transformation
print(reduced_strips)

[[39500.         140.        -390.95       781.25    ]
 [39501.         140.        -385.1        781.25    ]
 [39502.         140.        -385.         781.25    ]
 ...
 [38329.        -389.881213  -119.        -165.      ]
 [38330.        -389.981213  -119.        -165.      ]
 [38331.        -395.831226  -119.        -165.      ]]


In [11]:
# Iterate through modules (groups of 32 entries (one per SiPM))
# Find the limits of the SiPM positions
# Use these limits to infer the strip orientation

for i in range(24):
    limits = [ 999999, -999999, 999999, -999999, 999999, -999999]
    for j in range(32):
        if(reduced_strips[i*32 + j, 1] < limits[0]): limits[0] = reduced_strips[i*32 + j, 1]
        if(reduced_strips[i*32 + j, 1] > limits[1]): limits[1] = reduced_strips[i*32 + j, 1]
        if(reduced_strips[i*32 + j, 2] < limits[2]): limits[2] = reduced_strips[i*32 + j, 2]
        if(reduced_strips[i*32 + j, 2] > limits[3]): limits[3] = reduced_strips[i*32 + j, 2]
        if(reduced_strips[i*32 + j, 3] < limits[4]): limits[4] = reduced_strips[i*32 + j, 3]
        if(reduced_strips[i*32 + j, 3] > limits[5]): limits[5] = reduced_strips[i*32 + j, 3]
    print("Strip ", reduced_strips[i*32, 0] / 100)
    if(limits[4] != limits[5]): print("What the f*ck")
    if(limits[4] > 0): print("Downstream")
    else: print("Upstream")
    if(limits[0] == limits[1]): 
        print("Horizontal")
        print("x: ", limits[0])
        print("y: ", limits[2], " to ", limits[3])
        print("z: ", limits[4])
    if(limits[2] == limits[3]): 
        print("Vertical")
        print("x: ", limits[0], " to ", limits[1])
        print("y: ", limits[2])
        print("z: ", limits[4])
    print()

Strip  395.0
Downstream
Horizontal
x:  140.0
y:  -390.95  to  -295.850011
z:  781.25

Strip  394.0
Downstream
Horizontal
x:  140.0
y:  -295.740009  to  -200.640057
z:  781.25

Strip  393.0
Downstream
Horizontal
x:  140.0
y:  -200.530063  to  -105.43003499999998
z:  781.25

Strip  379.0
Downstream
Vertical
x:  44.849805  to  139.95
y:  -105.370026
z:  781.25

Strip  381.0
Downstream
Vertical
x:  -50.36040000000003  to  44.739795000000015
y:  -105.370026
z:  781.25

Strip  380.0
Downstream
Vertical
x:  -145.570605  to  -50.470410000000015
y:  -105.370026
z:  781.25

Strip  404.0
Upstream
Horizontal
x:  189.0
y:  -390.95  to  -295.850011
z:  -165.0

Strip  403.0
Upstream
Horizontal
x:  189.0
y:  -295.740009  to  -200.640057
z:  -165.0

Strip  402.0
Upstream
Horizontal
x:  189.0
y:  -200.530063  to  -105.43003499999998
z:  -165.0

Strip  391.0
Upstream
Vertical
x:  365.850018  to  460.95
y:  -119.0
z:  -165.0

Strip  390.0
Upstream
Vertical
x:  270.640057  to  365.74001599999997
y:  -119.0